<a href="https://colab.research.google.com/github/Shawn-Varghese/big-data-challenge/blob/main/Video-game_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [21]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-29 20:37:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.53MB/s    in 0.2s    

2022-05-29 20:37:19 (4.53 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [23]:
#Putting s3 bucket and the data together
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-01-31 00:08:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [24]:
#number of games
df.count()

145431

In [25]:
#removed unnecessary points
df = df.dropna()
print(df.count())

145425


In [26]:
#review table
reviewdf = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
reviewdf.show(15)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193|2015-01-31 00:08:00|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535|2015-01-31 00:08:00|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344|2015-01-31 00:08:00|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204|2015-01-31 00:08:00|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561|2015-01-31 00:08:00|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192|2015-01-31 00:08:00|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395|2015-01-31 00:08:00|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568|2015-01-31 00:08:00|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568|2015-01-31 00:08:00|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568|2015-01-31 00:08:00|
|R3B3UHK1FO0ERS|    89268

In [27]:
reviewdf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [28]:
#product table 
productdf = df.select(["product_id","product_title"])
productdf = productdf.dropDuplicates()
productdf.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows



In [29]:
#grouping by customers and finding count of individual customers 
from pyspark.sql.functions import desc

customersdf = df.groupby("customer_id").agg({"customer_id":"count"})
customersdf = customersdf.orderBy(desc("count(customer_id)"))
customersdf = customersdf.withColumnRenamed("count(customer_id)", "customer_count") 
customersdf.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21925455|            95|
|   40955995|            92|
|   23564949|            83|
|   52520442|            73|
|   14279279|            72|
|   51520185|            65|
|   20751317|            54|
|   50529050|            49|
|   11584162|            49|
|   25575022|            49|
|   36527466|            48|
|   39942708|            48|
|   31012456|            46|
|   52919259|            45|
|   19371429|            43|
|   46544620|            43|
|   26151231|            42|
|   53080901|            42|
|   45688707|            40|
|   14222166|            39|
+-----------+--------------+
only showing top 20 rows



In [30]:
customersdf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [31]:
#vine table 
from pyspark.sql.functions import col

vinedf = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])


#vinedf = vinedf.filter(col("vine")  == "Y")
vinedf.show(50)


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSH1OZ87OYK92|          2|            2|          3|   N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|
| R3YOOS71KM5M9|          5|            0|          0|   N|
|R3R14UATT3OUFU|          5|            0|          0|   N|
| RV2W9SGDNQA2C|          5|            0|          0|   N|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|
|R1LRYU1V0T3O38|          4|            0|          0|   N|
| R44QKV6FE5CJ2|          5|            0|          0|   N|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|
|R1JEEW4C6R89BA|          5|            0|          0|   N|
|R3B3UHK1FO0ERS|          1|            0|          0|   N|
|R2GVSDHW513SS1|          5|            0|          0|   N|
|R1R1NT516PYT73|          5|            0|          0|   N|
|R3K624QDQKENN9|          4|            

In [32]:
vinedf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [33]:
#connecting to AWS
mode="append"
jdbc_url = "jdbc:postgresql://XXXX/videogames_db"
config = {"user": "XXXX",
          "password": "XXXX",
          "driver":"org.postgresql.Driver"}


In [ ]:
reviewdf.write.jdbc(url= jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
productdf.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
customersdf.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [38]:
vinedf.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)